In [1]:
import os
import time
import sys
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import numpy as np

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# helper method to define model
def deepnn(x):
    x_image = tf.reshape(x, [-1, 28, 28, 1])
#     x_image = tf.image.resize_image_with_crop_or_pad(x_image, target_height=32, target_width=32)

    W_conv1 = weight_variable([5, 5, 1, 6])
    b_conv1 = bias_variable([6])
    h_conv1 = tf.nn.relu(conv2d(x_image, W_conv1) + b_conv1)

    h_pool1 = max_pool_2x2(h_conv1)

    W_conv2 = weight_variable([5, 5, 6, 16])
    b_conv2 = bias_variable([16])
    h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)

    h_pool2 = max_pool_2x2(h_conv2)

    W_fc1 = weight_variable([7*7*16, 120])
    b_fc1 = bias_variable([120])

    h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*16])
    h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

#     keep_prob = tf.placeholder(tf.float32)
#     h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

    W_fc2 = weight_variable([120, 10])
    b_fc2 = bias_variable([10])

    y_conv = tf.matmul(h_fc1, W_fc2)
              
#     logits = tf.nn.softmax(y_conv)
    return x_image, y_conv


def conv2d(x, W):
    return tf.nn.conv2d(x, W, strides=[1, 1, 1, 1], padding='SAME')


def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')


def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)


def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [4]:
#load graph and data and run training

tf.reset_default_graph()

# reading data
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

# defining TF model
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])
x_image, y_conv = deepnn(x)

cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = y_conv, labels = y_))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y_conv,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# trainning CNN
sess= tf.Session()

sess.run(tf.global_variables_initializer())
with sess.as_default():
    for i in range(10000):
        batch = mnist.train.next_batch(50)
        if i%10 == 0:
            train_accuracy = accuracy.eval(feed_dict={x:batch[0], y_: batch[1]})
            print("step %d, training accuracy %g"%(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0], y_: batch[1]})
        
    x_feed = mnist.test.images.reshape(-1, 784)[:100]
    y_feed = mnist.test.labels.reshape(-1, 10)[:100]
    print("test accuracy %g"%accuracy.eval(feed_dict={x: x_feed, y_: y_feed, 
                                                      }))

    test_x = mnist.test.images.reshape(-1, 784)[0]
    test_y = mnist.test.images.reshape(-1, 10)[0]
    feed_dict = {x: test_x, y:test_y}

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
step 0, training accuracy 0.08
step 10, training accuracy 0.08
step 20, training accuracy 0.16
step 30, training accuracy 0.1
step 40, training accuracy 0.14
step 50, training accuracy 0.3
step 60, training accuracy 0.4
step 70, training accuracy 0.36
step 80, training accuracy 0.5
step 90, training accuracy 0.54
step 100, training accuracy 0.5
step 110, training accuracy 0.56
step 120, training accuracy 0.38
step 130, training accuracy 0.54
step 140, training accuracy 0.6
step 150, training accuracy 0.58
step 160, training accuracy 0.64
step 170, training accuracy 0.7
step 180, training accuracy 0.58
step 190, training accuracy 0.74
step 200, training accuracy 0.66
step 210, training accuracy 0.6
step 220, training accuracy 0.7
step 230, training accuracy 0.68
step 240, training accuracy 0.72
s

KeyboardInterrupt: 

In [ ]:
def forwardprop(sess, image):
    image = image.reshape(-1, 784)
    output = sess.run(y_conv, feed_dict={x:image})
    return output

In [ ]:
test_x = mnist.test.images.reshape(-1, 784)[2]
test_y = mnist.test.labels.reshape(-1, 10)[2]
output = forwardprop(sess, test_x)

In [ ]:
test_y

In [ ]:
label = [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]

In [ ]:
def create_adv_image(x_image, y_label, lr=0.01, n_steps=17):
    
    original_image = x_image
    
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=y_label, logits=y_conv)
    deriv = tf.gradients(loss, x)
    image_adv = tf.stop_gradient(x - tf.sign(deriv)*lr)
    image_adv = tf.clip_by_value(image_adv, 0, 1)
    
    for _ in range(n_steps):
        dydx = sess.run(deriv, {x:x_image.reshape(-1, 784)})
        
        x_adv = sess.run(image_adv, {x: x_image.reshape(-1, 784)})
        
        x_image = np.reshape(x_adv, (1, 784))
        
    return x_image

In [ ]:
adv = create_adv_image(test_x, label, n_steps=12)

In [ ]:
op = forwardprop(sess, adv)

In [ ]:
op

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
fig, ax = plt.subplots()

im = ax.imshow(adv.reshape((28, 28)), cmap='jet')

fig.colorbar(im, ax=ax)

# plt.imshow(adv.reshape((28, 28)))
plt.savefig('adversarial_1_output_0')

In [ ]:
adv = adv.reshape((784, ))

In [ ]:
diff = adv - test_x

In [ ]:
fig, ax = plt.subplots()

im = ax.imshow(diff.reshape((28, 28)), cmap='jet')

fig.colorbar(im, ax=ax)

# plt.imshow(diff.reshape((28, 28)))
plt.savefig('./difference_image')

In [ ]:
fig, ax = plt.subplots()

im = ax.imshow(test_x.reshape((28, 28)), cmap='jet')

fig.colorbar(im, ax=ax)

# plt.imshow(diff.reshape((28, 28)))
plt.savefig('./actual_image')

In [ ]:
saver = tf.train.Saver()

saver.save(sess, './adversarial_example')